In [1]:
import numpy as np
import pandas as pd

class files:
    g_forecasts = './grouped_test_forecasts.csv'
    g_errors = './grouped_train_errors.csv'
    b_forecasts = './base_test_forecasts.csv'
    b_errors = './base_train_errors.csv'

## Covariance and Shrinkage

In [7]:
g_errors = pd.read_csv(files.g_errors).drop('type', axis=1)
b_errors = pd.read_csv(files.b_errors)
g_errors['date'] = pd.to_datetime(g_errors['date'])
b_errors['date'] = pd.to_datetime(b_errors['date'])
g_errors = g_errors.sort_values(['id', 'date']).reset_index(drop=True)
b_errors = b_errors.sort_values(['unique_id', 'date']).reset_index(drop=True)
display(g_errors)
display(b_errors)

,id,date,error
0,Apple,2020-08-01,195.360476
1,Apple,2020-08-02,-1295.187063
2,Apple,2020-08-03,-887.507978
3,Apple,2020-08-04,112.970402
4,Apple,2020-08-05,729.860833
...,...,...,...
527462,total,2024-05-29,-1976.925425
527463,total,2024-05-30,-21827.830454
527464,total,2024-05-31,-20617.067602
527465,total,2024-06-01,-6200.176423


,unique_id,date,error
0,1,2022-06-13,40.676338
1,1,2022-06-14,70.852570
2,1,2022-06-15,94.430732
3,1,2022-06-16,-52.711509
4,1,2022-06-17,-54.597530
...,...,...,...
3477121,5431,2024-05-29,12.355801
3477122,5431,2024-05-30,-14.809783
3477123,5431,2024-05-31,34.907267
3477124,5431,2024-06-01,-27.324684


In [ ]:
from tqdm import tqdm
uids = b_errors['unique_id'].unique().tolist()
dfids = []
for id in tqdm(uids):
    dfids.append(b_errors.loc[b_errors['unique_id'] == id, ['date', 'error']].set_index('date'))

gids = g_errors['id'].unique().tolist()
for id in tqdm(gids):
    dfids.append(g_errors.loc[g_errors['id'] == id, ['date', 'error']].set_index('date'))

error_df = pd.concat(dfids, axis=1)
error_df.columns = uids + gids

100%|██████████| 421/421 [00:13<00:00, 31.86it/s]


In [12]:
sample_cov = error_df.cov()

In [133]:
import numpy as np

sample_cov = sample_cov.fillna(0)
def mint_shrinkage(S: pd.DataFrame):
    S_np = S.to_numpy()
    D_np = np.diag(np.diag(S_np))
    lamb = np.sum((S_np - D_np)**2) / np.sum(S_np ** 2)
    print(lamb)
    S_sh = (1 - lamb) * S_np + lamb * D_np
    return pd.DataFrame(S_sh, columns=S.columns, index=S.index)

shrunk_cov = mint_shrinkage(sample_cov)

0.5488524210845198


In [17]:
shrunk_cov.to_csv('./shrunk_cov.csv')

In [134]:
inv_shrunk_cov = np.linalg.inv(shrunk_cov.to_numpy())

## Summing Matrix S

In [101]:
from preprocessing import get_inventory
inventory = get_inventory()
inventory['city'] = inventory['warehouse'].apply(lambda x: x.split('_')[0])
inventory = inventory.drop(['product_unique_id', 'warehouse', 'L24', 'L23', 'L34'], axis=1)
inventory = inventory.sort_values('unique_id').reset_index(drop=True)
cats = inventory.T

summer = pd.DataFrame(np.zeros((len(uids+gids), len(uids))),
                      index=uids+gids, columns=uids)
for uid in uids:
    rows = cats[uid].to_list() + ['total']
    summer.loc[rows, uid] = 1

display(summer)

,1,2,3,7,9,10,11,12,15,18,...,5421,5422,5423,5424,5425,5427,5428,5429,5430,5431
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Watermelon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Wrap,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yogurt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zucchini,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Formatting Predictions

In [123]:
g_fors = pd.read_csv(files.g_forecasts).drop('type', axis=1)
b_fors = pd.read_csv(files.b_forecasts)
g_fors['date'] = pd.to_datetime(g_fors['date'])
b_fors['date'] = pd.to_datetime(b_fors['date'])
g_fors = g_fors.sort_values(['id', 'date']).reset_index(drop=True)
b_fors = b_fors.sort_values(['unique_id', 'date']).reset_index(drop=True)
b_fors['w_sales'] = b_fors['weight'] * b_fors['sales']

g_fors = g_fors.groupby(['id', 'date']).mean().reset_index()

In [124]:
ids = b_fors['unique_id'].unique().tolist()
dfs = []
for id in tqdm(ids):
    dfs.append(b_fors.loc[b_fors['unique_id'] == id, ['date', 'w_sales']].set_index('date'))
    
fors = pd.concat(dfs, axis=1)
fors.columns = ids

for id in tqdm(ids):
    fors[id] = fors[id].fillna(fors[id].mean())

  0%|          | 0/3625 [00:00<?, ?it/s]

100%|██████████| 3625/3625 [00:00<00:00, 4808.05it/s]


In [125]:
fors = fors.T
display(fors)

date,2024-06-03,2024-06-04,2024-06-05,2024-06-06,2024-06-07,2024-06-08,2024-06-09,2024-06-10,2024-06-11,2024-06-12,2024-06-13,2024-06-14,2024-06-15,2024-06-16
1,527.890252,581.007600,514.146933,593.511378,604.255909,513.830193,484.184500,605.006834,565.288619,496.422772,578.258188,613.136058,539.197577,519.257091
2,1346.869771,1076.632547,1053.992670,1262.389634,1551.709003,1021.543511,925.937139,1364.488288,1036.747818,1071.004072,1329.527414,1446.790891,1179.667361,996.723563
3,200.268412,157.180811,203.282290,205.414349,211.260341,218.733598,191.141106,220.135028,201.845653,198.410124,211.117643,200.856078,194.604334,197.735325
7,73.896797,50.212477,49.995431,61.422501,74.122373,55.745748,26.343209,60.628411,49.474231,54.274569,67.614262,83.945157,57.428239,27.256442
9,117.639714,119.984944,128.141889,135.482086,166.772067,125.153388,114.722293,133.505900,128.061745,125.510105,117.249716,137.880254,123.041893,96.987774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5427,58.328511,61.660043,61.922625,59.590460,71.794695,68.669349,57.672274,59.812044,53.922412,59.523419,57.835656,72.655079,63.653880,54.281759
5428,117.604964,104.523494,111.217946,148.897723,156.625691,110.615262,143.118640,4452.695756,4186.059656,4407.867806,4634.538351,4483.863341,3697.782487,2058.108547
5429,146.244359,138.475726,144.613287,214.414323,245.141820,140.798563,145.845206,6639.522049,6403.944567,6339.669021,6782.071398,6957.989432,4583.428977,4482.873332
5430,41.630739,40.858940,26.942574,33.941139,54.491236,34.494209,34.724363,1467.536319,1344.732546,1359.886975,1473.979849,1484.378896,1242.390550,1098.648849


In [126]:
ids = g_fors['id'].unique().tolist()
dfs = []
for id in tqdm(ids):
    dfs.append(g_fors.loc[g_fors['id'] == id, ['date', 'sales']].set_index('date'))

gfors = pd.concat(dfs, axis=1)
gfors.columns = ids

display(gfors)

  0%|          | 0/421 [00:00<?, ?it/s]

100%|██████████| 421/421 [00:00<00:00, 648.80it/s]


,Apple,Apricot,Arugula,Asparagus,Avocado,Bag,Bagel,Baguette,Bakery,Bakery Snack,...,Tortilla,Toust,Turkey,Vegetable,Waffle,Watermelon,Wrap,Yogurt,Zucchini,total
date,,,,,,,,,,,,,,,,,,,,,
2024-06-03,30044.881135,4233.747548,2797.284497,13020.451350,23193.834205,1182.451513,1358.279724,8218.142341,86054.215265,1520.993858,...,4362.992711,15625.411256,12494.961810,5284.603647,552.860006,32753.447210,821.579342,14.612902,8286.239381,1.006611e+06
2024-06-04,29136.311479,4260.352001,2865.559697,19517.872371,22683.946462,1333.865540,1195.053484,8474.878565,79732.998182,1839.987148,...,4331.057945,15779.875025,12337.176359,5412.558752,576.170344,33565.584148,810.712392,10.000539,8972.006885,9.635527e+05
2024-06-05,28559.094691,4542.589705,2779.983330,19131.744926,23682.422682,639.720833,1212.205251,8010.103764,75573.625050,1854.027989,...,4534.967848,15672.732761,12317.308256,5522.556057,570.450427,31071.105172,940.278383,11.780064,8885.293246,9.676859e+05
2024-06-06,27162.132780,5133.453140,2922.701108,17439.115958,22456.866568,2062.023700,1289.132701,10548.640320,97464.929267,1733.691660,...,4544.382454,16423.944798,13374.199924,6078.902816,611.288474,31255.158006,960.647899,18.301530,9057.999545,1.066021e+06
2024-06-07,34826.380119,5761.014999,2821.815074,17953.784517,25422.569467,2340.668466,1443.039298,10968.704132,107890.398662,1999.775556,...,5475.041433,18531.599272,15827.272198,7300.267193,871.898238,35137.705317,971.023875,160.458341,10605.606216,1.243706e+06
2024-06-08,26588.910514,4476.887938,2485.601340,16243.181949,21764.766398,1836.685003,1130.940019,8650.303014,84254.977199,1800.331247,...,4437.382967,15275.539544,11762.903103,5650.909040,612.353820,27879.918801,835.174060,35.234356,8800.585082,9.533756e+05
2024-06-09,29247.270256,4596.704184,2656.875502,19215.698798,23587.610991,1477.374469,1344.932228,9859.416801,68413.660479,1905.554182,...,4592.692614,15332.542031,11134.349801,5549.163421,620.334189,31584.851967,1052.564671,28.154501,8139.380141,8.905404e+05
2024-06-10,28502.217014,4842.603700,2506.847668,17628.112140,21975.610068,1471.795803,1416.791138,8093.745425,72562.541083,1714.576115,...,4701.541943,15545.100449,12764.861051,5332.652421,587.481443,28827.170617,736.728000,51.237590,8849.445634,1.007486e+06
2024-06-11,29778.670998,4451.333568,2831.406576,14427.805108,23197.070391,716.994358,1351.840409,8389.430617,80448.913742,1771.992015,...,4540.725770,15550.447249,11813.078060,5634.848315,607.524653,28300.548828,977.352609,37.560999,8312.363342,9.629212e+05


In [127]:
gfors = gfors.T

In [129]:
baseforecasts = pd.concat([fors, gfors])
baseforecasts

date,2024-06-03,2024-06-04,2024-06-05,2024-06-06,2024-06-07,2024-06-08,2024-06-09,2024-06-10,2024-06-11,2024-06-12,2024-06-13,2024-06-14,2024-06-15,2024-06-16
1,5.278903e+02,581.007600,514.146933,5.935114e+02,6.042559e+02,513.830193,484.184500,6.050068e+02,565.288619,496.422772,5.782582e+02,6.131361e+02,539.197577,519.257091
2,1.346870e+03,1076.632547,1053.992670,1.262390e+03,1.551709e+03,1021.543511,925.937139,1.364488e+03,1036.747818,1071.004072,1.329527e+03,1.446791e+03,1179.667361,996.723563
3,2.002684e+02,157.180811,203.282290,2.054143e+02,2.112603e+02,218.733598,191.141106,2.201350e+02,201.845653,198.410124,2.111176e+02,2.008561e+02,194.604334,197.735325
7,7.389680e+01,50.212477,49.995431,6.142250e+01,7.412237e+01,55.745748,26.343209,6.062841e+01,49.474231,54.274569,6.761426e+01,8.394516e+01,57.428239,27.256442
9,1.176397e+02,119.984944,128.141889,1.354821e+02,1.667721e+02,125.153388,114.722293,1.335059e+02,128.061745,125.510105,1.172497e+02,1.378803e+02,123.041893,96.987774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Watermelon,3.275345e+04,33565.584148,31071.105172,3.125516e+04,3.513771e+04,27879.918801,31584.851967,2.882717e+04,28300.548828,31276.913232,3.129877e+04,3.488070e+04,30618.231521,25888.530886
Wrap,8.215793e+02,810.712392,940.278383,9.606479e+02,9.710239e+02,835.174060,1052.564671,7.367280e+02,977.352609,815.877315,8.534374e+02,1.233764e+03,827.458528,824.923814
Yogurt,1.461290e+01,10.000539,11.780064,1.830153e+01,1.604583e+02,35.234356,28.154501,5.123759e+01,37.560999,42.004023,4.474470e+01,1.708886e+02,27.518824,24.075217
Zucchini,8.286239e+03,8972.006885,8885.293246,9.058000e+03,1.060561e+04,8800.585082,8139.380141,8.849446e+03,8312.363342,8352.929816,9.353438e+03,1.092225e+04,8464.506039,8111.814130


In [82]:
## All Together

In [135]:
S_np = summer.to_numpy()
inv = np.linalg.inv(S_np.transpose() @ inv_shrunk_cov @ S_np)
M = S_np @ inv @ S_np.transpose() @ inv_shrunk_cov

In [136]:
aggforecasts = baseforecasts.copy()
for col in baseforecasts.columns.to_list():
    aggforecasts[col] = M @ baseforecasts[col].to_numpy()

In [137]:
display(aggforecasts)
display(baseforecasts)

date,2024-06-03,2024-06-04,2024-06-05,2024-06-06,2024-06-07,2024-06-08,2024-06-09,2024-06-10,2024-06-11,2024-06-12,2024-06-13,2024-06-14,2024-06-15,2024-06-16
1,2.175735e+03,1.800510e+03,1.809691e+03,2.019743e+03,1.857184e+03,1.663560e+03,1.558429e+03,1.355085e+03,1.284378e+03,1.274940e+03,9.023428e+02,5.446394e+02,8.760269e+02,1301.010715
2,5.202884e+03,4.920168e+03,4.175098e+03,5.953054e+03,6.131329e+03,4.430282e+03,4.669568e+03,5.534730e+03,5.650960e+03,5.786709e+03,6.156715e+03,5.926765e+03,5.206380e+03,5390.337145
3,2.779316e+02,2.197608e+02,2.416460e+02,1.377597e+02,9.824228e+01,2.467438e+02,3.392349e+02,5.589682e+01,1.530131e+02,6.441684e+01,1.204520e+02,1.260946e+02,1.410788e+02,221.030941
7,2.142307e+01,1.086410e+01,1.692317e+01,8.545380e+00,2.010132e+01,2.426470e+01,1.182267e+01,2.455625e+01,1.149106e+01,6.202915e+00,2.828817e+01,7.669814e+01,3.496304e+01,4.323553
9,4.032284e+02,3.590805e+02,3.540328e+02,4.995108e+02,4.966990e+02,3.579882e+02,2.784219e+02,4.025911e+02,3.630015e+02,4.095545e+02,3.794644e+02,3.027565e+02,3.281953e+02,346.600969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Watermelon,3.131442e+04,3.891575e+04,4.092899e+04,3.692315e+04,4.877706e+04,3.194166e+04,2.801324e+04,3.460525e+04,2.793819e+04,2.981996e+04,3.684649e+04,4.796088e+04,3.276215e+04,24255.761129
Wrap,-2.814025e+01,1.239908e+02,9.513067e+01,1.797156e+01,4.311185e+02,-2.093689e+00,-2.245392e+02,5.981895e+02,2.510301e+02,4.175486e+02,4.946699e+02,8.081837e+02,3.686795e+02,56.019023
Yogurt,-8.986973e+01,-6.567369e+00,-4.296985e+01,-1.341661e+02,-5.636311e+01,-3.075616e+01,-3.373960e+01,-4.994037e+01,-7.488926e+01,-9.372059e+01,-5.577364e+01,4.931282e+01,-3.977855e+01,-64.351099
Zucchini,1.120549e+04,1.011956e+04,9.487787e+03,9.036685e+03,1.058332e+04,9.110853e+03,9.151647e+03,8.106381e+03,8.339075e+03,8.277902e+03,9.536382e+03,1.035947e+04,9.183679e+03,8790.063288


date,2024-06-03,2024-06-04,2024-06-05,2024-06-06,2024-06-07,2024-06-08,2024-06-09,2024-06-10,2024-06-11,2024-06-12,2024-06-13,2024-06-14,2024-06-15,2024-06-16
1,5.278903e+02,581.007600,514.146933,5.935114e+02,6.042559e+02,513.830193,484.184500,6.050068e+02,565.288619,496.422772,5.782582e+02,6.131361e+02,539.197577,519.257091
2,1.346870e+03,1076.632547,1053.992670,1.262390e+03,1.551709e+03,1021.543511,925.937139,1.364488e+03,1036.747818,1071.004072,1.329527e+03,1.446791e+03,1179.667361,996.723563
3,2.002684e+02,157.180811,203.282290,2.054143e+02,2.112603e+02,218.733598,191.141106,2.201350e+02,201.845653,198.410124,2.111176e+02,2.008561e+02,194.604334,197.735325
7,7.389680e+01,50.212477,49.995431,6.142250e+01,7.412237e+01,55.745748,26.343209,6.062841e+01,49.474231,54.274569,6.761426e+01,8.394516e+01,57.428239,27.256442
9,1.176397e+02,119.984944,128.141889,1.354821e+02,1.667721e+02,125.153388,114.722293,1.335059e+02,128.061745,125.510105,1.172497e+02,1.378803e+02,123.041893,96.987774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Watermelon,3.275345e+04,33565.584148,31071.105172,3.125516e+04,3.513771e+04,27879.918801,31584.851967,2.882717e+04,28300.548828,31276.913232,3.129877e+04,3.488070e+04,30618.231521,25888.530886
Wrap,8.215793e+02,810.712392,940.278383,9.606479e+02,9.710239e+02,835.174060,1052.564671,7.367280e+02,977.352609,815.877315,8.534374e+02,1.233764e+03,827.458528,824.923814
Yogurt,1.461290e+01,10.000539,11.780064,1.830153e+01,1.604583e+02,35.234356,28.154501,5.123759e+01,37.560999,42.004023,4.474470e+01,1.708886e+02,27.518824,24.075217
Zucchini,8.286239e+03,8972.006885,8885.293246,9.058000e+03,1.060561e+04,8800.585082,8139.380141,8.849446e+03,8312.363342,8352.929816,9.353438e+03,1.092225e+04,8464.506039,8111.814130
